## 서울 아파트 분양가 분석

In [ ]:
# library import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# csv read
first_file_name = '전국 평균 분양가격(2013년 9월부터 2015년 8월까지).csv' #2013~2015.08
last_file_name = '주택도시보증공사_전국 평균 분양가격(2019년 12월).csv' #2015.09~2019.12

df_first = pd.read_csv(first_file_name, encoding="euc-kr") # utf-8
df_last = pd.read_csv(last_file_name, encoding="euc-kr")

display(df_first, df_last)

In [ ]:
df_first.info()

In [ ]:
df_last.info()

In [ ]:
df_last.isna().sum()

In [ ]:
df_last['분양가격'] = pd.to_numeric(df_last['분양가격(㎡)'], errors='coerce')

# errors
# - ignore: 변경할 수 없는 데이터면 -> 원본 데이터 그대로 반환
# - coerce: 변경할 수 없는 데이터면 -> NaN 으로 반환.
# - raise: numeric으로 변경할 수 없는 경우 오류가 발생하며 작업 중단

In [ ]:
df_last['평당분양가격'] = df_last['분양가격'] * 3.3 # (1평=> 3.3m2)

In [ ]:
df_last.head(5)

In [ ]:
df_first.head(5)

In [ ]:
df_last['분양가격'].describe()

In [ ]:
df_last['규모구분'].unique()

In [ ]:
# 규모구분 데이터에서 
# (1) 전용면적,이하 문자열을 제거하고
# (2) 초과 문자열을 ~로 변경
# (3) 빈 문자열 제거
# array(['전체', '60㎡', '60㎡~85㎡', '85㎡~102㎡', '102㎡~'], dtype=object)

df_last['전용면적'] = df_last['규모구분'].str.replace("전용면적", "")
df_last['전용면적'] = df_last['전용면적'].str.replace("이하", "")
df_last['전용면적'] = df_last['전용면적'].str.replace("초과", "~")
df_last['전용면적'] = df_last['전용면적'].str.replace(" ", "")

df_last['전용면적']

In [ ]:
df_last = df_last.drop(['분양가격(㎡)', '규모구분'], axis=1)
df_last

In [ ]:
# 지역별 평당분양가격의 기술통계값을 보고싶다.
df_last.groupby(["지역명"])['평당분양가격'].describe()

In [ ]:
# 전용면적별 평당분양가격의 평균값은 어떻게 될까?
df_last.groupby(['전용면적'])['평당분양가격'].mean()

In [ ]:
# 지역별 전용면적의 평당분양가격 평균값을 df로 나타내시오.
df_last.groupby(['지역명', '전용면적'])['평당분양가격'].mean().unstack()

In [ ]:
# 지역명이 인덱스, 연도별이 컬럼 -> 연도별 평당 분양가격의 평균값은?

df_last.groupby(['연도', '지역명'])['평당분양가격'].mean().unstack().transpose()

In [ ]:
df_last_pivot = pd.pivot_table(df_last, index=['전용면적'], columns=['연도'], values=['평당분양가격'])

In [ ]:
df_last_pivot['평당분양가격'][2015]

In [ ]:
plt.rcParams['font.family'] = "AppleGothic" # Mac

In [ ]:
df_last.groupby(['전용면적'])['평당분양가격'].mean().sort_values().plot.bar()

In [ ]:
df_last.groupby(['연도'])['평당분양가격'].mean().plot()

In [ ]:
df_last.pivot_table(index='월', columns=['연도', '전용면적'], values='평당분양가격').plot.box()

In [ ]:
# 연도별 월에 따른 평당분양가격 변화 추이
df_last.pivot_table(index="월", columns=["연도"], values="평당분양가격").plot(figsize=(20,5), rot=90)

In [ ]:
import seaborn as sns

In [ ]:
df_last_sorted = df_last.sort_values(by='평당분양가격', ascending=False)
sns.barplot(data=df_last_sorted, x='지역명', y="평당분양가격")

In [ ]:
sns.boxplot(data=df_last, x="연도", y="평당분양가격", hue="전용면적")

In [ ]:
max_price = df_last['평당분양가격'].max()

In [ ]:
df_last[df_last['평당분양가격'] == max_price]

In [ ]:
df_first

In [ ]:
df_first

In [ ]:
df_last

In [ ]:
# 가로 방향의 데이터 프레임 => 세로 방향으로 전환
df_first_melt = df_first.melt(id_vars='지역', var_name='기간', value_name="평당분양가격")
df_first_melt

In [ ]:
# 2013년1월
# 2013년2월

date = "2013년12월"

# 숫자 데이터만 표현
year = date.split("년")[0]

month = date.split("년")[1].replace("월", "")
print(year, month)

In [ ]:
def parse_year(date):
    year = date.split("년")[0]
    return year

def parse_month(date):
    month = date.split("년")[1].replace("월", "")
    return month

In [ ]:
# 기간 컬럼에서 => 년, 월로 데이터를 각각 컬럼 별로 분리
df_first_melt['연도'] = df_first_melt['기간'].apply(parse_year)
df_first_melt['월'] = df_first_melt['기간'].apply(parse_month)

In [ ]:
# df_last에서 전용면적이 '전체'인 데이터만 가져오시오.
# 단, 컬럼은 지역명, 연도, 월, 평당분양가격만 가져오시오.
df_last[df_last['전용면적'] == "전체"][['지역명', '연도', '월', '평당분양가격']]

df_last_prepare = df_last.loc[df_last['전용면적'] == "전체", ['지역명', '연도', '월', '평당분양가격']]

df_last_prepare

In [ ]:
df_first_melt